<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Introduction to Pandas</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">


## Overview:

* **Teaching:** 30 minutes
* **Exercises:** 30 minutes

### Questions
1. What is Pandas?
1. What are the basic Pandas data structures?
1. How can I read data into Pandas?
1. What are some of the data operations available in Pandas?

### Objectives
1. <a href="#series">Data Series</a>
1. <a href="#frames">Data Frames</a>
1. <a href="#loading">Loading Data in Pandas</a>
1. <a href="#missing">Missing Data</a>
1. <a href="#manipulating">Manipulating Data</a>

<a name="series"></a>
## Data Series
Data series are one of the fundamental data structures in Pandas. You can think of them like a dictionary; they have a key (index) and value (data/values) like a dictionary, but also have some handy functionality attached to them.

To start out, let's create a series from scratch. We'll imagine these are temperature observations.

In [ ]:
import pandas as pd
temperatures = pd.Series([23, 20, 25, 18])
temperatures

The values on the left are the index (zero based integers by default) and on the right are the values. Notice that the data type is an integer. Any NumPy datatype is acceptable in a series.

That's great, but it'd be more useful if the station were associated with those values. In fact you could say we want the values *indexed* by station name.

In [ ]:
temperatures = pd.Series([23, 20, 25, 18], index=['TOP', 'OUN', 'DAL', 'DEN'])
temperatures

Now, very similar to a dictionary, we can use the index to access and modify elements.

In [ ]:
temperatures['DAL']

In [ ]:
temperatures[['DAL', 'OUN']]

We can also do basic filtering, math, etc.

In [ ]:
temperatures[temperatures > 20]

In [ ]:
temperatures + 2

Remember how I said that series are like dictionaries? We can create a series striaght from a dictionary.

In [ ]:
dps = {'TOP': 14,
       'OUN': 18,
       'DEN': 9,
       'PHX': 11,
       'DAL': 23}

dewpoints = pd.Series(dps)
dewpoints

It's also easy to check and see if an index exists in a given series:

In [ ]:
'PHX' in dewpoints

In [ ]:
'PHX' in temperatures

Series have a name attribute and their index has a name attribute.

In [ ]:
temperatures.name = 'temperature'
temperatures.index.name = 'station'

In [ ]:
temperatures

### Exercise
* Create a series of pressures for stations TOP, OUN, DEN, and DAL (assign any values you like).
* Set the series name and series index name.
* Print the pressures for all stations which have a dewpoint below 15.

In [ ]:
# Your code goes here


#### Solution

In [ ]:
# %load solutions/make_series.py


<a href="#top">Top</a>
<hr style="height:2px;">

<a name="frames"></a>
## Data Frames
Series are great, but what about a bunch of related series? Something like a table or a spreadsheet? Enter the data frame. A data frame can be thought of as a dictionary of data series. They have indexes for their rows and their columns. Each data series can be of a different type , but they will all share a common index.

The easiest way to create a data frame by hand is to use a dictionary.

In [ ]:
data = {'station': ['TOP', 'OUN', 'DEN', 'DAL'],
        'temperature': [23, 20, 25, 18],
        'dewpoint': [14, 18, 9, 23]}

df = pd.DataFrame(data)
df

You can access columns (data series) using dictionary type notation or attribute type notation.

In [ ]:
df['temperature']

In [ ]:
df.dewpoint

Notice the index is shared and that the name of the column is attached as the series name.

You can also create a new column and assign values. If I only pass a scalar it is duplicated.

In [ ]:
df['wspeed'] = 0.
df

Let's set the index to be the station.

In [ ]:
df.index = df.station
df

Well, that's close, but we now have a redundant column, so let's get rid of it.

In [ ]:
df.drop('station', 1, inplace=True)
df

We can also add data and order it by providing index values. Note that the next cell contains data that's "out of order" compared to the dataframe shown above. However, by providing the index that corresponds to each value, the data is organized correctly into the dataframe.

In [ ]:
df['pressure'] = pd.Series([1010,1000,998,1018], index=['DEN','TOP','DAL','OUN'])
df

Now let's get a row from the dataframe instead of a column.

In [ ]:
df.loc['DEN']

We can even transpose the data easily if we needed that do make things easier to merge/munge later.

In [ ]:
df.T

Look at the `values` attribute to access the data as a 1D or 2D array for series and data frames recpectively.

In [ ]:
df.values

In [ ]:
df.temperature.values

### Exercise
* Add a series of rain observations to the existing data frame.
* Apply an instrument correction of -2 to the dewpoint observations.

In [ ]:
# Your code goes here


#### Solution

In [ ]:
# %load solutions/rain_obs.py


<a href="#top">Top</a>
<hr style="height:2px;">

<a name="loading"></a>
## Loading Data in Pandas
The real power of pandas is in manupulating and summarizing large sets of tabular data. To do that, we'll need a large set of tabular data. We've included a file in this directory called `JAN17_CO_ASOS.txt` that has all of the ASOS observations for several stations in Colorado for January of 2017. It's a few hundred thousand rows of data in a tab delimited format. Let's load it into Pandas.

In [ ]:
df = pd.read_table('Jan17_CO_ASOS.txt')

In [ ]:
df.head()

In [ ]:
df = pd.read_table('Jan17_CO_ASOS.txt', parse_dates=['valid'])

In [ ]:
df.head()

In [ ]:
df = pd.read_table('Jan17_CO_ASOS.txt', parse_dates=['valid'], na_values='M')

In [ ]:
df.head()

Let's look in detail at those column names. Turns out we need to do some cleaning of this file. Welcome to real world data analysis.

In [ ]:
df.columns

In [ ]:
df.columns = ['station', 'time', 'temperature', 'dewpoint', 'pressure']

In [ ]:
df.head()

For other formats of data CSV, fixed width, etc. that are tools to read it as well. You can even read excel files straight into Pandas.

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="missing"></a>
## Missing Data
We've already dealt with some missing data by turning the 'M' string into actual NaN's while reading the file in. We can do one better though and delete any rows that have all values missing. There are similar operations that could be performed for columns. You can even drop if any values are missing, all are missing, or just those you specify are missing.

In [ ]:
len(df)

In [ ]:
df.dropna(axis=0, how='all', subset=['temperature', 'dewpoint', 'pressure'], inplace=True)

In [ ]:
len(df)

In [ ]:
df.head()

### Exercise
Our dataframe `df` has data in which we dropped any entries that were missing all of the temperature, dewpoint and pressure observations. Let's modify our command some and create a new dataframe `df2` that only keeps observations that have all three variables (i.e. if a pressure is missing, the whole entry is dropped). This is useful if you were doing some computation that requires a complete observation to work.

In [ ]:
# Your code goes here
# df2 = 

#### Solution

In [ ]:
# %load solutions/drop_obs.py


Lastly, we still have the original index values. Let's reindex to a new zero-based index for only the rows that have valid data in them.

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="manipulating"></a>
## Manipulating Data
We can now take our data and do some intersting things with it. Let's start with a simple min/max.

In [ ]:
print(f'Min: {df.temperature.min()}\nMax: {df.temperature.max()}')

You can also do some useful statistics on data with attached methods like corr for correlation coefficient.

In [ ]:
df.temperature.corr(df.dewpoint)

We can also call a `groupby` on the data frame to start getting some summary information for each station.

In [ ]:
df.groupby('station').mean()

### Exercise
Calculate the min, max, and standard deviation of the temperature field grouped by each station.

In [ ]:
# Calculate min


In [ ]:
# Calculate max


In [ ]:
# Calculate standard deviation


#### Solution

In [ ]:
# %load solutions/calc_stats.py


Now, let me show you how to do all of that and more in a single call.

In [ ]:
df.groupby('station').describe()

Now let's suppose we're going to make a meteogram or similar and want to get all of the data for a single station.

In [ ]:
df.groupby('station').get_group('0CO').head().reset_index(drop=True)

### Exercise
* Round the temperature column to whole degrees.
* Group the observations by temperaturee and use the count method to see how many instances of the rounded temperatures there are in the dataset.

In [ ]:
# Your code goes here


#### Solution

In [ ]:
# %load solutions/temperature_count.py


<a href="#top">Top</a>
<hr style="height:2px;">